In [16]:
from pymupdf4llm import LlamaMarkdownReader

# Define the reader object
llama_reader = LlamaMarkdownReader()

# Load the data
indexed_chunks = llama_reader.load_data("../data/raw/sample.pdf")

Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[================

In [25]:
from openai import OpenAI

# Define the OpenAI client
client = OpenAI(base_url="http://172.29.3.249:1234/v1", api_key="lm-studio")

# Define the prompt
completion = client.chat.completions.create(
    model="bartowski/Llama-3-8B-Instruct-Gradient-1048k-GGUF",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that can generate questions and answers from the provided text. Present the questions and answers in a structured JSON format. Output should only be the JSON.",
        },
        {
            "role": "user",
            "content": "Generate 3 questions and answers from the provided text.",
        },
        {"role": "user", "content": indexed_chunks[0].text},
    ],
    temperature=0.2,
    response_format={"type": "json_object"},
)

# Print the response
print(completion.choices[0].message.content.strip("</s>"))

Here are three questions based on the provided text:

1. What is the significance of weights in multi-criteria decision making models?

assistant: Weights provide information about the relative importance of considered criteria.

2. Why is using ranks to elicit weights by formulas more reliable than directly assigning weights to criteria?

assistant: Because decision makers are usually more confident about the ranks of some criteria than their weights, and they can agree on ranks more easily.

3. What advantages do methods that rely only on ordinal information about attribute importance have over other methods?

assistant: They can be used in situations where there is time pressure, quality nature of criteria, lack of knowledge, imprecise or incomplete information, decision maker’s limited attention and information processing capability.
